In [489]:
#!pip install yahoo_fin --upgrade

In [490]:
import requests
import pandas as pd
import json
import math
import requests
import urllib.parse
from datetime import datetime, timedelta

import yahoo_fin
from yahoo_fin.stock_info import get_data, get_quote_table, get_quote_data
from pandas_datareader import data as pdr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [491]:
# formatting on dates
date_interval = "{0},{1}".format(datetime.now().date() - timedelta(days=2),datetime.now().date() + timedelta(days=1))

In [492]:
# handling the cookies 
def load_cookies():
	headers = {
	    'authority': 'www.barchart.com',
	    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
	    'sec-ch-ua-mobile': '?0',
	    'sec-ch-ua-platform': '"Windows"',
	    'upgrade-insecure-requests': '1',
	    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
	    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
	    'sec-fetch-site': 'none',
	    'sec-fetch-mode': 'navigate',
	    'sec-fetch-user': '?1',
	    'sec-fetch-dest': 'document',
	    'accept-language': 'en-GB,en;q=0.9',
	}
	response = requests.get('https://www.barchart.com/options/volatility-percent-change/increase', headers=headers)
	return response.cookies

def get_parent(url,params,ignore,col_names,pagination):
	cookies = load_cookies()
	headers = {
	    'authority': 'www.barchart.com',
	    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
	    'accept': 'application/json',
	    'x-xsrf-token': urllib.parse.unquote(cookies.get('XSRF-TOKEN')),
	    'sec-ch-ua-mobile': '?0',
	    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36',
	    'sec-ch-ua-platform': '"Windows"',
	    'sec-fetch-site': 'same-origin',
	    'sec-fetch-mode': 'cors',
	    'sec-fetch-dest': 'empty',
	    'referer': url,
	    'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
	    'cookie': ';'.join(["{0}={1}".format(each_cookie[0],urllib.parse.unquote(each_cookie[1])) for each_cookie in cookies.items()])
			}

	# parsing data into pandas
	final_data_list = []
	response = requests.get(url, headers=headers, params=params)
	total_pages = math.ceil(json.loads(response.content).get("total") / 100)
	
	for page in range(1,total_pages+1):
		new_params = []
		for each_param in params:
			if 'page' != each_param[0]:
				new_params.append(each_param)
			else:
				new_params.append(('page', str(page)))
		params = new_params
		response = requests.get(url, headers=headers, params=params)
		final_data_list.extend(json.loads(response.content).get("data"))
		if not pagination:
			break
	final_data_list = [{key:val for key,val in row.items() if key not in ignore} for row in final_data_list]
	data_frame = pd.DataFrame(final_data_list)
	data_frame.columns = col_names
	return data_frame

**1) Most Active Stock Options**
<br>obtain latest data from: [barchart.com](https://www.barchart.com/options/most-active/stocks?orderBy=optionsTotalVolume&orderDir=desc)

In [493]:
# Most Active Stock Options

params = [
  ('list', 'options.mostActive.us'),
  ('fields', 'symbol,symbolType,symbolName,hasOptions,lastPrice,priceChange,percentChange,optionsImpliedVolatilityRank1y,optionsTotalVolume,optionsPutVolumePercent,optionsCallVolumePercent,optionsPutCallVolumeRatio,tradeTime,symbolCode'),
  ('orderBy', 'optionsTotalVolume'),
  ('orderDir', 'desc'),
  ('between(lastPrice,.10,)', ''),
  ('between(tradeTime,{0})'.format(date_interval), ''),
  ('meta', 'field.shortName,field.type,field.description'),
  ('hasOptions', 'true'),
  ('page', '1'),
  ('limit', '100'),
  ('raw', '1')]

high_vol_stock_options = get_parent('https://www.barchart.com/proxies/core-api/v1/quotes/get',params,['symbolCode','raw','symbolType','hasOptions'],
		                    ["Symbol","Name","Last","Change","%Chg","IV Rank","Options Vol" ,"% Put","% Call","Put/Call Vol","Time"],True)

In [494]:
# Highest vol stock options clean up
high_vol_stock_options['Time'] = high_vol_stock_options['Time'].astype('datetime64')
to_num = ['Last','IV Rank', 'Options Vol', '% Put', '% Call', 'Put/Call Vol']
high_vol_stock_options[to_num] = high_vol_stock_options[to_num].replace(r'\D+', '', regex=True)
high_vol_stock_options[to_num] = high_vol_stock_options[to_num].astype('float')
high_vol_stock_options[['Last', 'IV Rank', 'Put/Call Vol']] = high_vol_stock_options[['Last', 'IV Rank', 'Put/Call Vol']]/100
high_vol_stock_options[['% Put', '% Call']] = high_vol_stock_options[['% Put', '% Call']]/10

In [495]:
high_vol_stock_options.head()

,Symbol,Name,Last,Change,%Chg,IV Rank,Options Vol,% Put,% Call,Put/Call Vol,Time
0,TSLA,Tesla Inc,775.22,-0.26,-0.03%,21.21,1462621.0,58.2,41.8,1.39,2021-10-01
1,AAPL,Apple Inc,142.65,+1.15,+0.81%,27.42,1358844.0,32.2,67.8,0.47,2021-10-01
2,AMC,AMC Entertainment Holdings Inc,38.46,+0.40,+1.05%,2.92,699780.0,24.6,75.4,0.33,2021-10-01
3,MRK,Merck & Company,81.40,+6.29,+8.37%,51.37,395863.0,20.9,79.1,0.26,2021-10-01
4,AMZN,Amazon.com Inc,3283.26,-1.78,-0.05%,34.95,350640.0,42.0,58.0,0.72,2021-10-01


**2) Most Active ETF Options**
<br>obtain latest data from: [barchart.com](https://www.barchart.com/options/most-active/etfs?orderBy=optionsTotalVolume&orderDir=desc)

In [496]:
# Most Active ETFs

params = [
  ('list', 'options.mostActive.etf'),
	('fields', 'symbol,symbolType,symbolName,hasOptions,lastPrice,priceChange,percentChange,optionsImpliedVolatilityRank1y,optionsTotalVolume,optionsPutVolumePercent,optionsCallVolumePercent,optionsPutCallVolumeRatio,tradeTime,symbolCode'),
	('orderBy', 'optionsTotalVolume'),
	('orderDir', 'desc'),
	('between(lastPrice,.10,)', ''),
	('between(tradeTime,{0})'.format(date_interval), ''),
	('meta', 'field.shortName,field.type,field.description'),
	('hasOptions', 'true'),
	('page', '1'),
	('limit', '100'),
	('raw', '1')]

high_vol_etf_options = get_parent('https://www.barchart.com/proxies/core-api/v1/quotes/get',params,['symbolCode','raw','symbolType','hasOptions'],
		                  ["Symbol","Name","Last","Change","%Chg","IV Rank","Options Vol" ,"% Put","% Call","Put/Call Vol","Time"],True)

In [497]:
# Highest vol etr options clean up
high_vol_etf_options['Time'] = high_vol_etf_options['Time'].astype('datetime64')
etf_to_num = ['Last','IV Rank', 'Options Vol', '% Put', '% Call', 'Put/Call Vol']
high_vol_etf_options[etf_to_num] = high_vol_etf_options[etf_to_num].replace(r'\D+', '', regex=True)
high_vol_etf_options[etf_to_num] = high_vol_etf_options[etf_to_num].astype('float')
high_vol_etf_options[['Last', 'IV Rank', 'Put/Call Vol']] = high_vol_etf_options[['Last', 'IV Rank', 'Put/Call Vol']]/100
high_vol_etf_options[['% Put', '% Call']] = high_vol_etf_options[['% Put', '% Call']]/10

In [498]:
high_vol_etf_options.head()

,Symbol,Name,Last,Change,%Chg,IV Rank,Options Vol,% Put,% Call,Put/Call Vol,Time
0,SPY,S&P 500 SPDR,434.24,+5.10,+1.19%,27.69,4905866.0,54.7,45.3,1.21,2021-10-01
1,QQQ,Nasdaq QQQ Invesco ETF,360.18,+2.22,+0.62%,30.14,1277885.0,57.1,42.9,1.33,2021-10-01
2,IWM,Russell 2000 Ishares ETF,222.28,+3.53,+1.61%,27.33,686273.0,73.0,27.0,2.71,2021-10-01
3,XLE,S&P 500 Energy Sector SPDR,53.84,+1.75,+3.36%,23.49,295938.0,14.2,85.8,0.17,2021-10-01
4,HYG,High Yield Corp Bond Ishares Iboxx $ ETF,87.41,-0.08,-0.09%,11.29,251628.0,86.1,13.9,6.17,2021-10-01


**3) Highest Implied Volatility Options**
<br>obtain latest data from: [barchart.com](https://www.barchart.com/options/highest-implied-volatility)

In [499]:
# Highest implied volatility table

params = [
  ('fields', 'symbol,baseSymbol,baseLastPrice,baseSymbolType,symbolType,strikePrice,expirationDate,daysToExpiration,bidPrice,midpoint,askPrice,lastPrice,volume,openInterest,volumeOpenInterestRatio,volatility,tradeTime,symbolCode,hasOptions'),
  ('orderBy', 'volatility'),
  ('baseSymbolTypes', 'stock'),
  ('between(lastPrice,.10,)', ''),
  ('between(daysToExpiration,15,)', ''),
  ('between(tradeTime,{0})'.format(date_interval), ''),
  ('orderDir', 'desc'),
  ('between(volatility,60,)', ''),
  ('limit', '200'),
  ('between(volume,500,)', ''),
  ('between(openInterest,100,)', ''),
  ('in(exchange,(AMEX,NASDAQ,NYSE))', ''),
  ('meta', 'field.shortName,field.type,field.description'),
  ('hasOptions', 'true'),
  ('raw', '1')]
  
highest_volitility = get_parent('https://www.barchart.com/proxies/core-api/v1/options/get',params,['symbol','baseSymbolType','raw'],
		                ["Symbol","Price","Type","Strike","Exp Date","DTE","Bid","Midpoint","Ask","Last","Volume","Open Int","Vol/OI","IV","Last Trade"],False)

In [500]:
highest_volitility.head()

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,Vol/OI,IV,Last Trade
0,NXTD,0.4020,Call,0.5000,11/19/21,49,0.1000,0.1300,0.1500,0.1400,"2,325","1,519",1.53,295.35%,10/01/21
1,PROG,1.7900,Call,2.0000,11/19/21,49,0.5500,0.5800,0.6000,0.6000,"7,415","1,965",3.77,262.77%,10/01/21
2,PROG,1.7900,Call,1.5000,11/19/21,49,0.7000,0.7300,0.7500,0.7300,"3,904","7,206",0.54,247.26%,10/01/21
3,XENE,15.60,Put,7.50,11/19/21,49,0.65,1.03,1.40,0.70,"3,036","2,181",1.39,240.68%,09/30/21
4,BBIG,6.14,Put,20.00,01/21/22,112,15.00,15.15,15.30,15.00,"2,145","5,634",0.38,237.66%,09/30/21


In [501]:
# Highest IV data clean up
highest_volitility[['Exp Date', 'Last Trade']] = highest_volitility[['Exp Date', 'Last Trade']].astype('datetime64')
to_num = ['Price', 'Strike', 'DTE', 'Bid', 'Midpoint', 'Ask', 'Last',	'Volume',	'Open Int',	'IV', 'Vol/OI']
highest_volitility['Volume'] = highest_volitility['Volume'].str.replace(',', '')
highest_volitility['Open Int'] = highest_volitility['Open Int'].str.replace(',', '')
highest_volitility['IV'] = highest_volitility['IV'].str.replace('%', '')
highest_volitility[to_num] = highest_volitility[to_num].astype('float')

**3) Highest Implied Volatility Options Screener**
<br>Screening Criteria:
*   Implied Volatility > 100%
*   Day to expiry within 21 days
*   Stock price < USD 100 per share





In [502]:
# screen for highest IV, within 21 days out for put options
high_iv_screener = highest_volitility.query('IV> 100 & DTE<22 & Price< 100')
high_iv_screener

,Symbol,Price,Type,Strike,Exp Date,DTE,Bid,Midpoint,Ask,Last,Volume,Open Int,Vol/OI,IV,Last Trade
9,AMC,38.46,Call,95.0,2021-10-22,21.0,0.46,0.48,0.49,0.48,1574.0,4625.0,0.34,214.34,2021-10-01
12,DMYI,10.40,Call,20.0,2021-10-15,15.0,0.10,0.13,0.15,0.15,1563.0,8807.0,0.18,206.20,2021-09-30
23,BBIG,6.14,Call,7.0,2021-10-22,21.0,0.75,0.78,0.80,0.80,556.0,1845.0,0.30,189.65,2021-10-01
39,DMYI,10.40,Call,15.0,2021-10-15,15.0,0.25,0.30,0.35,0.30,862.0,7534.0,0.11,167.36,2021-09-30
45,DMYI,10.40,Call,14.0,2021-10-15,15.0,0.40,0.43,0.45,0.40,641.0,3738.0,0.17,163.39,2021-09-30
52,DMYI,10.40,Call,12.5,2021-10-15,15.0,0.50,0.55,0.60,0.60,4511.0,7527.0,0.60,153.06,2021-09-30
62,AMC,38.46,Call,60.0,2021-10-22,21.0,0.85,0.89,0.92,0.89,680.0,1340.0,0.51,149.97,2021-10-01
81,DMYI,10.40,Call,11.0,2021-10-15,15.0,0.80,0.88,0.95,0.93,1622.0,2736.0,0.59,139.90,2021-09-30
86,DMYI,10.40,Call,10.0,2021-10-15,15.0,1.25,1.30,1.35,1.34,1971.0,5948.0,0.33,137.28,2021-09-30
94,MNKD,4.52,Call,5.0,2021-10-22,21.0,0.35,0.40,0.45,0.34,2008.0,3121.0,0.64,134.65,2021-10-01


In [503]:
# put tickers to a list
highest_iv_list = high_iv_screener['Symbol'].unique().tolist()

**4) High Volume Stock Options Screener**
<br>Screening Criteria:
*   Implied Volatility > 10%
*   Options Volume (transaction) > 20k
*   Stock price < USD 100 per share





In [504]:
# screen for highest IV, within 21 days out for put options
high_vol_options_screener = high_vol_stock_options.query('Last< 100 & `Options Vol`> 20000 & `IV Rank` >10')
high_vol_options_screener

,Symbol,Name,Last,Change,%Chg,IV Rank,Options Vol,% Put,% Call,Put/Call Vol,Time
3,MRK,Merck & Company,81.40,+6.29,+8.37%,51.37,395863.0,20.9,79.1,0.26,2021-10-01
9,CSCO,Cisco Systems Inc,55.14,+0.71,+1.30%,12.99,237538.0,7.1,92.9,0.08,2021-10-01
12,AAL,American Airlines Gp,21.65,+1.13,+5.51%,12.62,222661.0,35.8,64.2,0.56,2021-10-01
16,CLF,Cleveland-Cliffs Inc,19.66,-0.15,-0.76%,25.60,168457.0,31.3,68.7,0.46,2021-10-01
17,BAC,Bank of America Corp,43.08,+0.63,+1.48%,25.36,167424.0,45.4,54.6,0.83,2021-10-01
...,...,...,...,...,...,...,...,...,...,...,...
162,MOS,Mosaic Company,37.80,+2.08,+5.82%,33.19,21158.0,21.0,79.0,0.27,2021-10-01
164,CAN,Canaan Inc ADR,6.41,+0.30,+4.91%,13.73,21012.0,3.2,96.8,0.03,2021-10-01
165,MS,Morgan Stanley,99.05,+1.74,+1.79%,45.68,20961.0,33.5,66.5,0.50,2021-10-01
167,BOX,Box Inc,24.81,+1.14,+4.82%,28.75,20404.0,4.6,95.4,0.05,2021-10-01


In [505]:
# put tickers to a list
high_vol_stock_options_list = high_vol_options_screener['Symbol'].unique().tolist()

**5) High Volume ETF Options Screener**
<br>Screening Criteria:
*   Implied Volatility > 10%
*   Options Volume (transaction) > 20k
*   Stock price < USD 100 per share





In [506]:
# screen for highest IV, within 21 days out for put options
high_vol_etf_screener = high_vol_etf_options.query('Last< 100 & `Options Vol`> 20000 & `IV Rank` >10')
high_vol_etf_screener

,Symbol,Name,Last,Change,%Chg,IV Rank,Options Vol,% Put,% Call,Put/Call Vol,Time
3,XLE,S&P 500 Energy Sector SPDR,53.84,+1.75,+3.36%,23.49,295938.0,14.2,85.8,0.17,2021-10-01
4,HYG,High Yield Corp Bond Ishares Iboxx $ ETF,87.41,-0.08,-0.09%,11.29,251628.0,86.1,13.9,6.17,2021-10-01
5,EWZ,Brazil Ishares MSCI ETF,33.08,+0.95,+2.96%,34.89,210984.0,80.3,19.7,4.08,2021-10-01
6,VXX,Ipath.B S&P 500 VIX Short-Term Futures ETN,26.82,-1.01,-3.63%,56.16,189686.0,45.1,54.9,0.82,2021-10-01
7,UVXY,Trust Ultra VIX Short Term Futures ETF,23.21,-1.34,-5.46%,23.81,167622.0,45.6,54.4,0.84,2021-10-01
10,JETS,US Global Jets ETF,24.89,+1.26,+5.33%,27.45,130985.0,10.7,89.3,0.12,2021-10-01
13,EEM,Emerging Markets Ishares MSCI ETF,50.33,-0.05,-0.10%,29.92,123414.0,76.2,23.8,3.20,2021-10-01
14,XLF,S&P 500 Financials Sector SPDR,38.15,+0.62,+1.65%,16.48,113057.0,50.2,49.8,1.01,2021-10-01
16,FXI,China Large-Cap Ishares ETF,38.62,-0.31,-0.80%,51.28,100307.0,40.8,59.2,0.69,2021-10-01
17,EFA,EAFE Ishares MSCI ETF,78.20,+0.19,+0.24%,29.01,77846.0,50.2,49.8,1.01,2021-10-01


In [507]:
# put tickers to a list
high_vol_etf_options_list = high_vol_etf_screener['Symbol'].unique().tolist()

In [508]:
print(highest_iv_list) 
print(high_vol_stock_options_list)

['AMC', 'DMYI', 'BBIG', 'MNKD', 'RIDE']
['MRK', 'CSCO', 'AAL', 'CLF', 'BAC', 'F', 'DKNG', 'CCL', 'LCID', 'JD', 'PFE', 'XOM', 'INTC', 'SOFI', 'X', 'UBER', 'UAL', 'T', 'DAL', 'BBIG', 'SDC', 'LVS', 'RIDE', 'SNAP', 'OXY', 'ITUB', 'PINS', 'GM', 'WFC', 'FCX', 'C', 'PTON', 'DLTR', 'KSS', 'HUT', 'ET', 'PENN', 'ING', 'SPIR', 'LUV', 'MGM', 'LAUR', 'WYNN', 'VZ', 'ATER', 'CVS', 'TCOM', 'ATVI', 'BTBT', 'HPQ', 'BITF', 'CCJ', 'AA', 'HBI', 'COP', 'SAVA', 'MRO', 'PBR', 'OCGN', 'GOLD', 'AEZS', 'SPWR', 'DVN', 'RKLB', 'NUE', 'MAT', 'ORCL', 'KO', 'APRN', 'IGT', 'TWTR', 'AEO', 'MPC', 'WY', 'BP', 'GOGO', 'CMCSA', 'GILD', 'MOS', 'CAN', 'MS', 'BOX', 'UAA']


In [509]:
set(highest_iv_list).intersection(high_vol_stock_options_list)

{'BBIG', 'RIDE'}

**6) Yahoo Finance Technical Indicators**
<br>Screening Criteria:
*   10 SMA/ 50 SMA/ 250 SMA
*   Implied Volatility > 10%
*   Options Volume (transaction) > 20k
*   Stock price < USD 100 per share





**6.1) Stocks**

In [510]:
p_stocklist = high_vol_options_screener.copy().drop(['Change', '%Chg', 'Put/Call Vol', 'Time'], axis=1).reset_index()

In [512]:
SMA10 =[]
for ticker in high_vol_stock_options_list:
  SMA10.append(yahoo_fin.stock_info.get_data(ticker, interval='1d')['close'][-10:].mean())  

In [513]:
SMA50 =[]
for ticker in high_vol_stock_options_list:
  SMA50.append(yahoo_fin.stock_info.get_data(ticker, interval='1d')['close'][-50:].mean())

In [ ]:
SMA250 =[]
for ticker in high_vol_stock_options_list: 
  SMA250.append(yahoo_fin.stock_info.get_data(ticker, interval='1d')['close'][-250:].mean())

In [515]:
SMA = pd.DataFrame(list(zip(SMA10, SMA50, SMA250))).round(2)

In [516]:
p_stocklist = p_stocklist.join(SMA)
p_stocklist.rename({0:'SMA10', 1:'SMA50', 2: 'SMA250'}, axis=1, inplace=True)

In [517]:
p_stocklist.head(3)

,index,Symbol,Name,Last,IV Rank,Options Vol,% Put,% Call,SMA10,SMA50,SMA250
0,3,MRK,Merck & Company,81.40,51.37,395863.0,20.9,79.1,78.26,75.69,75.19
1,9,CSCO,Cisco Systems Inc,55.14,12.99,237538.0,7.1,92.9,55.69,56.86,49.35
2,12,AAL,American Airlines Gp,21.65,12.62,222661.0,35.8,64.2,20.90,20.19,19.07


In [518]:
buylist = p_stocklist.query('SMA10 > SMA50 & SMA10 > SMA250')

In [521]:
print('Number of stocks for consideration:', buylist.shape[0])
buylist

Number of stocks for consideration: 30


,index,Symbol,Name,Last,IV Rank,Options Vol,% Put,% Call,SMA10,SMA50,SMA250
0,3,MRK,Merck & Company,81.40,51.37,395863.0,20.9,79.1,78.26,75.69,75.19
2,12,AAL,American Airlines Gp,21.65,12.62,222661.0,35.8,64.2,20.90,20.19,19.07
4,17,BAC,Bank of America Corp,43.08,25.36,167424.0,45.4,54.6,41.72,40.65,35.80
5,19,F,Ford Motor Company,14.16,14.77,150095.0,29.2,70.8,13.74,13.43,11.74
7,22,CCL,Carnival Corp,26.09,11.78,121209.0,26.1,73.9,24.91,23.33,23.24
8,23,LCID,Lucid Group Inc,24.61,46.41,118576.0,31.6,68.4,25.34,22.70,20.47
11,27,XOM,Exxon Mobil Corp,60.93,18.09,107175.0,31.0,69.0,57.61,56.29,51.87
18,42,DAL,Delta Air Lines Inc,45.38,17.28,72771.0,38.0,62.0,42.74,40.62,42.09
19,44,BBIG,Vinco Ventures Inc,6.14,43.26,69910.0,18.9,81.1,6.49,5.36,2.85
23,48,SNAP,Snap Inc,75.25,32.95,66636.0,27.0,73.0,76.43,74.84,57.02


**6.2) ETF**

In [522]:
etf_list = high_vol_etf_screener.copy().drop(['Change', '%Chg', 'Put/Call Vol', 'Time'], axis=1).reset_index()

In [523]:
SMA_10 =[]
for ticker in high_vol_etf_options_list:
  SMA_10.append(yahoo_fin.stock_info.get_data(ticker, interval='1d')['close'][-10:].mean())  

In [524]:
SMA_50 =[]
for ticker in high_vol_etf_options_list:
  SMA_50.append(yahoo_fin.stock_info.get_data(ticker, interval='1d')['close'][-50:].mean())

In [525]:
SMA_250 =[]
for ticker in high_vol_etf_options_list: 
  SMA_250.append(yahoo_fin.stock_info.get_data(ticker, interval='1d')['close'][-250:].mean())

In [526]:
etf_SMA = pd.DataFrame(list(zip(SMA_10, SMA_50, SMA_250))).round(2)

In [527]:
etf_list = etf_list.join(etf_SMA)
etf_list.rename({0:'SMA10', 1:'SMA50', 2: 'SMA250'}, axis=1, inplace=True)

In [529]:
etf_buylist = etf_list.query('SMA10 > SMA50 & SMA10 > SMA250')

In [530]:
print('Number of stocks for consideration:', etf_buylist.shape[0])
etf_buylist

Number of stocks for consideration: 6


,index,Symbol,Name,Last,IV Rank,Options Vol,% Put,% Call,SMA10,SMA50,SMA250
0,3,XLE,S&P 500 Energy Sector SPDR,53.84,23.49,295938.0,14.2,85.8,50.92,49.16,45.73
1,4,HYG,High Yield Corp Bond Ishares Iboxx $ ETF,87.41,11.29,251628.0,86.1,13.9,87.72,87.68,86.94
5,10,JETS,US Global Jets ETF,24.89,27.45,130985.0,10.7,89.3,23.82,23.00,23.52
14,26,EWJ,Japan Ishares MSCI ETF,69.93,66.52,32320.0,18.6,81.4,71.82,69.74,67.52
15,28,UNG,US Natural Gas Fund,19.28,72.76,29965.0,34.7,65.3,18.52,15.77,11.69
17,32,USO,US Oil Fund,53.26,16.96,27200.0,23.5,76.5,51.63,48.89,40.94
